In [26]:
# 라이브러리
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [27]:
path = "C:/Users/gihun/To_raise_a_child_well/data/gihun/dataset/"

In [28]:
sum_df = pd.read_excel(path+'출산율모델.xlsx',index_col=0)
sum_df_ = sum_df.copy()

In [29]:
# 연도별 데이터로 해당 연도 결측치 값 채우기
sum_df_['고령화비율'].loc['2006-01-01'] = 9.3
sum_df_['고령화비율'].loc['2007-01-01'] = 9.6 # 원래는 9.9,
sum_df_['고령화비율'].loc['2022-12-01'] = 18.0
sum_df_['고령화비율'] = sum_df_['고령화비율'].interpolate()

In [30]:
# 2006-01,2007-01의 값을 2009년대비 2008년의 타겟 증가율을 대입해서 보간법을 사용해서 결측치 대체
sum_df_['출산아수/가임기여성의수'].at['2007-01-01'] = sum_df_['출산아수/가임기여성의수'].at['2008-01-01']*(sum_df_['출산아수/가임기여성의수'].at['2008-01-01']/sum_df_['출산아수/가임기여성의수'].at['2009-01-01'])
sum_df_['출산아수/가임기여성의수'].at['2006-01-01'] = sum_df_['출산아수/가임기여성의수'].at['2007-01-01']*(sum_df_['출산아수/가임기여성의수'].at['2008-01-01']/sum_df_['출산아수/가임기여성의수'].at['2009-01-01'])
sum_df_['출산아수/가임기여성의수'] = sum_df_['출산아수/가임기여성의수'].interpolate()

In [31]:
sum_df_.to_excel(path+'전국데이터프레임2006~2021.xlsx')

In [32]:
# 독립변수,종속변수 설정
X = sum_df_.drop(columns='출산아수/가임기여성의수').loc['2006-01-01':'2021-12-01'].values
y = sum_df_['출산아수/가임기여성의수'].loc['2006-01-01':'2021-12-01'].values
X.shape, y.shape

((192, 10), (192,))

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### 이혼율,혼인건수,지가변동률: 22년도 없음/고령화비율:2007까지/출산아수,가임기여성의수:2008~2021/

In [34]:
# 파라미터설정
param_grid = {'max_depth': [3, 4, 5, 6],'learning_rate': [0.1, 0.01, 0.001],'n_estimators': [50, 100, 200], 'subsample':[0.5,0.8,1.0], 'eta':[0.2,0.25,0.3], 'gamma':[0, 0.3, 0.6, 1.0]}
# max_depth: 나무의 최대 깊이, 이 값을 늘리면 과적합 생길수도 있음 default 6
# min_child_weight: child_node에 필요한 인스턴스 가중치의 최소합계, 이 값이 클 수록 보수적이고 편향이 증가하게 돼서 변동성이 작아짐 default 1
# gamma: 감마가 클수록 알고리즘이 보수적임 default 0
# subsample: 0.5로 설정하면 xgboost가 나무를 성장시키기 전에 훈련데이터의 절반을 무작위로 샘플링한다. 이것은 과적합을 방지할 수도 있다 default 1
# eta: 과적합을 방지하기 위해 업데이트에 사용되는 단계 크기 축소입니다. default 0.3
# 모델설정
xgb = XGBRegressor()

# 그리드서치 파라미터 생성
grid = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=6)
grid.fit(X_train_scaled, y_train)
# 출력
print("Best hyperparameters: ", grid.best_params_)
print("Best score: ", grid.best_score_)

# 제일 좋은 것으로 다시돌리기
best_xgb = XGBRegressor(max_depth=grid.best_params_['max_depth'],
                        learning_rate=grid.best_params_['learning_rate'],
                        n_estimators=grid.best_params_['n_estimators'])
best_xgb.fit(X_train_scaled, y_train)

# 평가하기 
train_score = best_xgb.score(X_train_scaled, y_train)
print("Train set score: ", train_score)
test_score = best_xgb.score(X_test_scaled, y_test)
print("Test set score: ", test_score)

Best hyperparameters:  {'eta': 0.2, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8}
Best score:  0.8194658319044295
Train set score:  0.8626843332518376
Test set score:  0.8938770504926383


In [35]:
print(sum_df_.drop(columns='출산아수/가임기여성의수').columns,best_xgb.feature_importances_)

Index(['실업률', '이혼율', '고령화비율', '혼인건수', '소비자물가지수', '아파트전세지수', '미분양주택지수',
       '은행예금지수', '경제활동참가율', '지가변동률'],
      dtype='object') [0.         0.         0.31088212 0.         0.12756833 0.1121667
 0.         0.27872664 0.17065616 0.        ]
